#### Домашнее задание.
## Урок 11 - Работа с файловой системой
#### Выполнил работу: Данил Кузнецов


In [1]:
import pandas as pd
import numpy as np

In [2]:
installs = pd.read_csv('les8_installs.csv',index_col = 0)
logins = pd.read_csv('les8_logins.csv',index_col = 0)
logins.reset_index(drop = True)

,client_id,event_timestamp,is_tester,is_youtuber
0,868610407601689670,2021-02-16 07:53:04,0,0
1,2434015632087848193,2021-02-16 07:51:23,0,0
2,2531632462363212905,2021-02-16 07:52:12,0,0
3,3205304356447992721,2021-02-16 07:54:17,0,0
4,7482763148151863115,2021-02-16 07:54:36,0,0
...,...,...,...,...
286173,374127069099732578,2021-02-15 12:56:25,0,0
286174,4905473983468376225,2021-02-15 12:57:54,0,0
286175,110174818069209226,2021-02-15 12:58:05,0,0
286176,590901015821838791,2021-02-15 12:58:33,0,0


### Перевожу тип данных

In [3]:
installs['install_date'] = pd.to_datetime(installs['install_date'], format='%Y-%m-%d')

In [4]:
logins['event_timestamp'] = pd.to_datetime(logins['event_timestamp'], format='%Y-%m-%d')
logins['event_timestamp'] = logins['event_timestamp'].dt.to_period('D').dt.start_time

### Удаляю дубликаты

In [5]:
logins.duplicated().sum()

163655

In [6]:
logins = logins.drop_duplicates().reset_index(drop=True)

## Импортирую свой собственный модуль, который вернет нам ролинг ретеншен

In [7]:
from my_molull322 import *


In [8]:
calculate_rolling_ret(installs,logins)

,lifetime,count_users,RRRT
0,0,28424,1.000000
1,1,12604,0.443428
2,2,10166,0.357656
3,3,8978,0.315860
4,4,8165,0.287257
...,...,...,...
131,131,80,0.002815
132,132,58,0.002041
133,133,42,0.001478
134,134,24,0.000844


# Подробное решение:

### Соединяю таблицы

In [9]:
df = installs.merge(logins, on='client_id', how='right')

### Нахожу первоначальную дату регистрации и последнюю дату захода

In [10]:
df = df.groupby('client_id',as_index = False).agg({'install_date':'min', 'event_timestamp':'max'})

### Вычисляю день жизни

In [11]:
df['lifetime'] = df['event_timestamp'] - df['install_date']
df['lifetime'] = df['lifetime'] / np.timedelta64(1, 'D')
df['lifetime'] = df['lifetime'].astype('int')
df.columns=['client_id','install_date','end_date','lifetime']
df

,client_id,install_date,end_date,lifetime
0,438136831630685,2021-01-04,2021-01-05,1
1,486804146468217,2021-01-27,2021-01-27,0
2,1158821161632327,2021-01-14,2021-01-14,0
3,1345466703683100,2021-01-08,2021-01-08,0
4,1388878700069838,2021-01-11,2021-01-13,2
...,...,...,...,...
28419,9221945498303309839,2021-01-20,2021-01-31,11
28420,9222067806632790758,2021-01-08,2021-01-10,2
28421,9222597237810583871,2021-01-07,2021-01-07,0
28422,9222670672591347665,2021-01-12,2021-01-12,0


### Вычисляю кол-во игроков в каждом дне

In [12]:
final_table = df.groupby('lifetime',as_index = False).agg({'client_id':'nunique'})
final_table.columns = ['lifetime','count_users']
final_table

,lifetime,count_users
0,0,15820
1,1,2438
2,2,1188
3,3,813
4,4,576
...,...,...
131,131,22
132,132,16
133,133,18
134,134,18


### тут неверное количество юзеров, изменим это с помощью цикла:

In [13]:
for i in range(df['lifetime'].min(), df['lifetime'].max()+1):
    final_table['count_users'][i] = df[df['lifetime']>=i]['client_id'].nunique()
final_table   

,lifetime,count_users
0,0,28424
1,1,12604
2,2,10166
3,3,8978
4,4,8165
...,...,...
131,131,80
132,132,58
133,133,42
134,134,24


In [14]:
final_table['RRRT'] = final_table['count_users'] / final_table['count_users'][0]
final_table

,lifetime,count_users,RRRT
0,0,28424,1.000000
1,1,12604,0.443428
2,2,10166,0.357656
3,3,8978,0.315860
4,4,8165,0.287257
...,...,...,...
131,131,80,0.002815
132,132,58,0.002041
133,133,42,0.001478
134,134,24,0.000844
